## Batched example 1

This notebook is the first of a series that shows how [GSSHA_Workflow.ipynb](../GSSHA_Workflow.ipynb) can be parameterized at the command line. This parameterization is designed to enable parameter sweeps, automatic report generation and other new uses for notebooks.

This first notebook aims to explain the basic mechanism used to parameterize notebooks. Although this is the least user friendly of the approaches, it also requires the fewest changes to EarthSim and the associated PyViz tools.

The rest of the notebook is largely the same as [GSSHA_Workflow.ipynb](../GSSHA_Workflow.ipynb) although there are a few changes that are used to expose the notebook parameters at the command line. The two parameters we choose to expose are ``rain_intensity`` and ``rain_duration``.


1. <a href="#Declare_nbparams">Declare the command line parameters</a>
2. <a href="#Display_nbparams">Display the notebook parameter widgets</a>
3. <a href="#Apply_nbparams">Apply notebook parameters and display</a>


Once the notebook is configured to use notebook parameters, they can be set at the command line as follows:

```bash
cd examples/topics/batched_example/
PARAM_JSON_INIT='{"rain_intensity":42, "rain_duration":50}' jupyter notebook GSSHA_Workflow_Batched_Example1.ipynb
```

You can also generate a report HTML file using ``nbconvert`` as follows:

```bash
PARAM_JSON_INIT='{"rain_intensity":25, "rain_duration":60}' jupyter nbconvert --execute GSSHA_Workflow_Batched_Example1.ipynb --output report1.html
```

This will set the ``rain_intensity`` and ``rain_duration`` values to ``42`` and ``50`` respectively from the default values of ``24`` and ``60``.

The ``PARAM_JSON_INIT`` variable can be used this way to set the notebook parameters in any context where the environment variables are available in the execution context of the notebook. One major limitation of this approach is that setting environment variables on Windows requires a different syntax.

In the next notebook, this approach for parameterizing notebooks is made more user friendly and generalized to work on Windows.

In [ ]:
from datetime import datetime, timedelta
import os
import glob

import param
import parambokeh
import numpy as np
import xarray as xr
import geoviews as gv
import holoviews as hv
import quest
import earthsim.gssha as esgssha
import earthsim.gssha.model as models
import cartopy.crs as ccrs

from earthsim.gssha import download_data, get_file_from_quest
from earthsim.gssha.model import UniformRoughness, CreateGSSHAModel
from holoviews.streams import PolyEdit, BoxEdit, PointDraw, CDSStream
from holoviews.operation.datashader import regrid, shade
from earthsim.io import save_shapefile, open_gssha, get_ccrs

regrid.aggregator = 'max'

hv.extension('bokeh')
%output holomap='scrubber' fps=2

In [ ]:
rm -r ./vicksburg_south/

## Declare the command line parameters <a id="Declare_nbparams"></a>

In this instance, the two chosen notebook parameters mirror those define on ``Simulation``. Mirroring parameters this way is optional and you can define any sensible notebook parameters that hook into the rest of the notebook workflow.

In [ ]:
class NotebookParams(param.Parameterized):
    
    rain_intensity= param.Number(default=24, bounds=(0,None), softbounds=(0,75))
    
    rain_duration = param.Integer(default=60,bounds=(0,None), softbounds=(0,1000000))

## Display the notebook parameter widgets  <a id="Display_nbparams"></a>

This step makes the notebook parameters available to change at the start of the notebook, parameterizing the interactive workflow. In addition, using ``initializer=parambokeh.JSONInit()`` allows these parameters to be set from the command line.

In [ ]:
parambokeh.Widgets(NotebookParams, initializer=parambokeh.JSONInit())

## Configure model parameters

In [ ]:
model_creator = esgssha.CreateGSSHAModel(name='Vicksburg South Model Creator',
                                        mask_shapefile='../../data/vicksburg_watershed/watershed_boundary.shp',
                                        grid_cell_size=90)
parambokeh.Widgets(model_creator)

### Setting the parameters of the ``roughness`` 

In [ ]:
model_creator.roughness = UniformRoughness()
parambokeh.Widgets(model_creator.roughness)

## Draw bounds to compute watershed

Allows drawing a bounding box and adding points to serve as input to compute a watershed:

In [ ]:
%%opts Polygons [width=900 height=500] (fill_alpha=0 line_color='black')
%%opts Points (size=10 color='red')
tiles = gv.WMTS('http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png',
                crs=ccrs.PlateCarree(), extents=(-91, 32.2, -90.8, 32.4))
box_poly = hv.Polygons([])
points = hv.Points([])
box_stream = BoxEdit(source=box_poly)
point_stream = PointDraw(source=points)
tiles * box_poly * points

In [ ]:
if box_stream.element:
    element = gv.operation.project(box_stream.element, projection=ccrs.PlateCarree())
    xs, ys = element.array().T
    bounds = (xs[0], ys[1], xs[2], ys[0])
    print("BOUNDS", bounds)
    
if point_stream.element:
    projected = gv.operation.project(point_stream.element, projection=ccrs.PlateCarree())
    print("COORDINATE:", projected.iloc[0]['x'][0], projected.iloc[0]['y'][0])

## Inspect and edit shapefile

The plot below allows editing the shapefile using a set of tools. The controls for editing are as follows:
    
* Double-clicking the polygon displays the vertices
* After double-clicking the point tool is selected and vertices can be dragged around
* By tapping on a vertex it can be selected, tapping in a new location while a single point is selected inserts a new vertex
* Multiple points can be selected by holding shift and then tapping or using the box_select tool
* Once multiple vertices are selected they can be deleted by selecting the point editing tool and pressing ``backspace``

In [ ]:
%%opts Shape [width=900 height=500 tools=['box_select']] (alpha=0.5)
mask_shape = gv.Shape.from_shapefile(model_creator.mask_shapefile).last
tiles = gv.WMTS('http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png')
vertex_stream = PolyEdit(source=mask_shape)
tiles * mask_shape

If any edits were made to the polygon in the plot above we save the ``watershed_boundary.shp`` back out and redisplay it to confirm our edits were applied correctly:

In [ ]:
%%opts Shape [width=600 height=400] (alpha=0.5)
if vertex_stream.data:
    edited_shape_fname = '../vicksburg_watershed_edited/watershed_boundary.shp'
    dir_name = os.path.dirname(edited_shape_fname)
    if not os.path.isdir(dir_name): os.makedirs(dir_name)
    save_shapefile(vertex_stream.data, edited_shape_fname, model_creator.mask_shapefile)
    model_creator.mask_shapefile = edited_shape_fname
    mask_shape = gv.Shape.from_shapefile(edited_shape_fname).last
mask_shape = mask_shape.opts() # Clear options
mask_shape

## Configure simulation parameters

In [ ]:
sim = esgssha.Simulation(name='Vicksburg South Simulation', simulation_duration=60*60,
                          rain_duration=30*60, model_creator=model_creator)

## Apply notebook parameters and display<a id="Apply_nbparams"></a>

This is the point at which the notebook parameters hook into the workflow. In this example, the two chosen parameters ``rain_duration`` and ``rain_intensity`` are simply set on ``sim``. In more complex examples, you may decide to compute the parameters set in the workflow as a function of the availabel notebook parameters.

In [ ]:
sim.rain_duration = NotebookParams.rain_duration 
sim.rain_intensity = NotebookParams.rain_intensity 

In [ ]:
parambokeh.Widgets(sim)

## Create the model

Note that the above code demonstrates how to collect user input, but it has not yet been connected to the remaining workflow, which uses code-based specification for the parameters.

In [ ]:
if sim.model_creator.project_name not in quest.api.get_collections():
    quest.api.new_collection(sim.model_creator.project_name)

In [ ]:
parambokeh.Widgets(sim.model_creator)

In [ ]:
# temporary workaround until workflow cleanup/parameterization is done
if sim.model_creator.project_name == 'test_philippines_small':
    sim.model_creator.roughness = models.GriddedRoughnessTable(
        land_use_grid=get_file_from_quest(sim.model_creator.project_name, sim.land_use_service, 'landuse', sim.model_creator.mask_shapefile),
        land_use_to_roughness_table='../philippines_small/land_cover_glcf_modis.txt')
else:    
    sim.model_creator.roughness = models.GriddedRoughnessID(
        land_use_grid=get_file_from_quest(sim.model_creator.project_name, sim.land_use_service, 'landuse', sim.model_creator.mask_shapefile),
        land_use_grid_id=sim.land_use_grid_id)

sim.model_creator.elevation_grid_path = get_file_from_quest(sim.model_creator.project_name, sim.elevation_service, 'elevation', sim.model_creator.mask_shapefile)

In [ ]:
model = sim.model_creator()

In [ ]:
# add card for max depth
model.project_manager.setCard('FLOOD_GRID',
                              '{0}.fgd'.format(sim.model_creator.project_name),
                              add_quotes=True)
# Add time-based depth grids to simulation
"""
See: http://www.gsshawiki.com/Project_File:Output_Files_%E2%80%93_Required

Filename or folder to output MAP_TYPE maps of overland flow depth (m) 
every MAP_FREQ minutes. If MAP_TYPE=0, then [value] is a folder name 
and output files are called "value\depth.####.asc" **
"""

model.project_manager.setCard('DEPTH', '.', add_quotes=True)
model.project_manager.setCard('MAP_FREQ', '1')

# add event for simulation (optional)
"""
model.set_event(simulation_start=sim.simulation_start,
                simulation_duration=timedelta(seconds=sim.simulation_duration),
                rain_intensity=sim.rain_intensity,
                rain_duration=timedelta(seconds=sim.rain_duration))
"""
# write to disk
model.write()

## Review model inputs

### Load inputs to the simulation

In [ ]:
name = sim.model_creator.project_name
CRS = get_ccrs(os.path.join(name, name+'_prj.pro'))

roughness_arr = open_gssha(os.path.join(name,'roughness.idx'))
msk_arr = open_gssha(os.path.join(name, name+'.msk'))
ele_arr = open_gssha(os.path.join(name, name+'.ele'))

roughness = gv.Image(roughness_arr, crs=CRS, label='roughness.idx')
mask = gv.Image(msk_arr, crs=CRS, label='vicksburg_south.msk')
ele  = gv.Image(ele_arr, crs=CRS, label='vicksburg_south.ele')

#### Shapefile vs. Mask

In [ ]:
tiles * regrid(mask) * mask_shape

#### Elevation

In [ ]:
tiles * regrid(ele) * mask_shape

#### Roughness

In [ ]:
tiles * regrid(roughness) * mask_shape

# Run Simulation

In [ ]:
from gsshapy.modeling import GSSHAFramework

In [ ]:
# TODO: how does the info here relate to that set earlier?

# TODO: understand comment below
# assuming notebook is run from examples folder
project_path = os.path.join(sim.model_creator.project_base_directory, sim.model_creator.project_name)
gr = GSSHAFramework("gssha",
                    project_path,
                    "{0}.prj".format(sim.model_creator.project_name),
                    gssha_simulation_start=sim.simulation_start,
                    gssha_simulation_duration=timedelta(seconds=sim.simulation_duration),
                    # load_simulation_datetime=True,  # use this if already set datetime params in project file
                   )

# http://www.gsshawiki.com/Model_Construction:Defining_a_uniform_precipitation_event
gr.event_manager.add_uniform_precip_event(sim.rain_intensity, 
                                          timedelta(seconds=sim.rain_duration))

gssha_event_directory = gr.run()

# Visualizing the outputs

### Load and visualize depths over time

In [ ]:
depth_nc = os.path.join(gssha_event_directory, 'depths.nc')
if not os.path.isfile(depth_nc):
    # Load depth data files
    depth_map = hv.HoloMap(kdims=['Minute'])
    for fname in glob.glob(os.path.join(gssha_event_directory, 'depth.*.asc')):
        depth_arr = open_gssha(fname)
        minute = int(fname.split('.')[-2])
        # NOTE: Due to precision issues not all empty cells match the NaN value properly, fix later
        depth_arr.data[depth_arr.data==depth_arr.data[0,0]] = np.NaN
        depth_map[minute] = hv.Image(depth_arr)

    # Convert data to an xarray and save as NetCDF
    arrays = []
    for minute, img in depth_map.items():
        ds = hv.Dataset(img)
        arr = ds.data.z.assign_coords(minute=minute)
        arrays.append(arr)
    depths = xr.concat(arrays, 'minute')
    depths.to_netcdf(depth_nc)
else:
    depths = xr.open_dataset(depth_nc)

depth_ds = hv.Dataset(depths)
depth_ds.data

Now that we have a Dataset of depths we can convert it to a series of Images.

In [ ]:
%%opts Image [width=600 height=400 logz=True xaxis=None yaxis=None] (cmap='viridis') Histogram {+framewise}
regrid(depth_ds.to(hv.Image, ['x', 'y'])).redim.range(z=(0, 0.04)).hist(bin_range=(0, 0.04))

We can also lay out the plots over time to allow for easier comparison.

In [ ]:
%%opts Image [width=300 height=300 logz=True xaxis=None yaxis=None] (cmap='viridis')
regrid(depth_ds.select(minute=range(10, 70, 10)).to(hv.Image, ['x', 'y']).redim.range(z=(0, 0.04))).layout().cols(3)

### Flood Grid Depth

(Maximum flood depth over the course of the simulation)

In [ ]:
%%opts Image [width=600 height=400] (cmap='viridis')
fgd_arr = open_gssha(os.path.join(gssha_event_directory,'{0}.fgd'.format(sim.model_creator.project_name)))
fgd = gv.Image(fgd_arr, crs=CRS, label='vicksburg_south.fgd').redim.range(z=(0, 0.04))
regrid(fgd, streams=[hv.streams.RangeXY]).redim.range(z=(0, 0.04))

### Analyzing the simulation speed

In [ ]:
%%opts Spikes [width=600]
times = np.array([os.path.getmtime(f) for f in glob.glob(os.path.join(gssha_event_directory, 'depth*.asc'))] )
minutes = (times-times[0])/60
hv.Spikes(minutes, kdims=['Real Time (minutes)'], label='Time elapsed for each minute of simulation time') +\
hv.Curve(np.diff(minutes), kdims=['Simulation Time (min)'], vdims=[('runtime', 'Runtime per minute simulation time')]).redim.range(runtime=(0, None))

Here if the "spikes" are regularly spaced, simulation time is regularly scaled with real time, and so you should be able read out the approximate time to expect per unit of simulation time.